In [ ]:
#PCA
import pandas as pd
import numpy
import altair as alt

In [ ]:
#PCA
pca = PCA(n_components=2)
pca.fit(P)
P2d = pca.transform(data.values)

Comece aqui

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Seq import Seq
import bioinfo

In [12]:
df = bioinfo.gen_df("./data/TRIM5ALPHA_complete_PopSet")

In [13]:
g = pd.DataFrame()

In [14]:
b = bioinfo.gen_df2(df['Species'][2],df['Sequence'][2],3)


In [15]:
#g.append(b)

In [25]:
#unindo todos em um msm plot

for i in range(16):
    df3 = (bioinfo.gen_df2(df['Species'][16],df['Sequence'][16],3)).append(bioinfo.gen_df2(df['Species'][i],df['Sequence'][i],3))


pandas.core.frame.DataFrame

In [27]:

type(bioinfo.PCA(df3))

sklearn.decomposition.pca.PCA

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Seq import Seq
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

lstamn = ['A','C','G','T']

list(itertools.product(lstamn,repeat=2))

[('A', 'A'),
 ('A', 'C'),
 ('A', 'G'),
 ('A', 'T'),
 ('C', 'A'),
 ('C', 'C'),
 ('C', 'G'),
 ('C', 'T'),
 ('G', 'A'),
 ('G', 'C'),
 ('G', 'G'),
 ('G', 'T'),
 ('T', 'A'),
 ('T', 'C'),
 ('T', 'G'),
 ('T', 'T')]

In [28]:
bioinfo.plot_pca(bioinfo.pca(df3))

ValueError: Shape of passed values is (3, 522), indices imply (3, 348)

In [29]:
#fazer pca com matriz união de todos os genes no msm data frame
#um plot para todos os data points do notebook PCA das 17 espécies